### Import Libraries

In [1]:
import os
import sys
os.environ['LD_LIBRARY_PATH'] = f"{sys.prefix}/lib"

import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import warnings

import utils  # Importa o utils.py do FMA que você já tem

2025-11-15 19:17:22.982722: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-15 19:17:23.046830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763248643.065734     854 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763248643.072606     854 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763248643.127588     854 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Audio Preprocessing

In [2]:
# --- Configuração de Paths ---
METADATA_DIR = '../fma_metadata'
# Este é o diretório que você criou com o script reorganize_audio_into_genres.py
AUDIO_DIR = '../fma_datasets/fma_small_genres' 
# Diretório para salvar os espectrogramas processados (MUITO IMPORTANTE)
SPECTROGRAM_DIR = '../preprocessed_spectrograms/30s/'

print(f"Metadados em: {METADATA_DIR}")
print(f"Áudio em: {AUDIO_DIR}")
print(f"Cache de Espectrogramas em: {SPECTROGRAM_DIR}")

# --- Parâmetros de Áudio e Espectrograma ---
SR = 22050
DURATION_SEC = 30
N_MELS = 128       # Número de bandas Mel (altura da imagem)
N_FFT = 2048
HOP_LENGTH = 512   # Salto entre frames
TARGET_SAMPLES = SR * DURATION_SEC

# Vamos calcular o shape da imagem (dimensão de tempo)
# n_frames = 1 + (TARGET_SAMPLES // HOP_LENGTH)
# No librosa 0.10+, é melhor usar librosa.time_to_frames
N_FRAMES = librosa.time_to_frames(DURATION_SEC, sr=SR, hop_length=HOP_LENGTH) + 1
SPEC_SHAPE = (N_MELS, N_FRAMES)

print(f"Taxa de Amostragem: {SR} Hz")
print(f"Shape do Espectrograma: {SPEC_SHAPE} (Mel-bins, Frames)")

Metadados em: ../fma_metadata
Áudio em: ../fma_datasets/fma_small_genres
Cache de Espectrogramas em: ../preprocessed_spectrograms/30s/
Taxa de Amostragem: 22050 Hz
Shape do Espectrograma: (128, np.int64(1292)) (Mel-bins, Frames)


In [3]:
# Carregar o tracks.csv usando o utils.py
tracks = utils.load(f'{METADATA_DIR}/tracks.csv')

# Filtrar apenas o fma_small
small_mask = tracks[('set', 'subset')] == 'small'
small_tracks = tracks.loc[small_mask]

# Codificar os gêneros (Labels)
label_encoder = LabelEncoder()
y_labels_str = small_tracks[('track', 'genre_top')]
y = label_encoder.fit_transform(y_labels_str)
class_names = label_encoder.classes_
N_CLASSES = len(class_names)

# Extrair IDs de faixa e IDs de artista (para o GroupKFold)
track_ids = small_tracks.index.to_numpy()
groups = small_tracks[('artist', 'id')].to_numpy()

print(f"Total de faixas 'fma_small': {len(track_ids)}")
print(f"Total de classes (gêneros): {N_CLASSES}")
print(f"Classes: {class_names}")

Total de faixas 'fma_small': 8000
Total de classes (gêneros): 8
Classes: ['Electronic' 'Experimental' 'Folk' 'Hip-Hop' 'Instrumental'
 'International' 'Pop' 'Rock']


In [4]:
def get_spectrogram_30s(track_id, genre_top):
    """
    Carrega um áudio, o normaliza para 30s exatos, extrai o Mel-espectrograma
    e o salva em cache como um arquivo .npy para reuso.
    """
    audio_path = f"{AUDIO_DIR}/{genre_top}/{track_id:06d}.mp3"
    cache_path = f"{SPECTROGRAM_DIR}/{genre_top}/{track_id:06d}.npy"
    
    # 1. Verificar se o cache já existe
    if os.path.exists(cache_path):
        try:
            return np.load(cache_path)
        except Exception as e:
            print(f"Cache corrompido para {track_id}, re-processando... ({e})")

    # 2. Se não existe, processar o áudio
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            y, sr_loaded = librosa.load(audio_path, sr=SR, duration=DURATION_SEC)
        
        # 3. Garantir que tenha exatamente 30s (661500 amostras)
        # Isso padroniza o input para a CNN
        y = librosa.util.fix_length(y, size=TARGET_SAMPLES)
        
        # 4. Extrair Mel-espectrograma
        S = librosa.feature.melspectrogram(y=y, sr=SR, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH)
        
        # 5. Converter para Log-Mel (dB)
        log_S = librosa.power_to_db(S, ref=np.max)
        
        # 6. Normalizar (Z-score) - Crucial para a CNN
        # Adiciona 1e-6 para evitar divisão por zero em silêncio
        log_S_normalized = (log_S - np.mean(log_S)) / (np.std(log_S) + 1e-6)

        # 7. Salvar em cache
        os.makedirs(os.path.dirname(cache_path), exist_ok=True)
        np.save(cache_path, log_S_normalized)
        
        return log_S_normalized

    except Exception as e:
        print(f"\nErro ao processar {audio_path}: {e}")
        return None

In [5]:
print("Iniciando processamento e carregamento de dados...")
print(f"Verificando cache em {SPECTROGRAM_DIR}")

# Listas para coletar metadados, NÃO os dados brutos
filepaths = []
y_data = []
groups_data = []
track_ids_data = []

# Usamos tqdm para ver o progresso
for track_id, genre_top, label, artist_id in tqdm(
    zip(track_ids, y_labels_str, y, groups), 
    total=len(track_ids),
    desc="Processando/Verificando faixas"
):
    
    spectrogram = get_spectrogram_30s(track_id, genre_top)
    
    if spectrogram is not None:
        # Garantir que o shape está correto
        if spectrogram.shape == SPEC_SHAPE:
            # Em vez de anexar o spectrogram, anexamos o CAMINHO DO CACHE
            cache_path = f"{SPECTROGRAM_DIR}/{genre_top}/{track_id:06d}.npy"
            filepaths.append(cache_path)
            y_data.append(label)
            groups_data.append(artist_id)
            track_ids_data.append(track_id)
        else:
            print(f"Skipping track {track_id}: shape incorreto {spectrogram.shape}")

# Converter para arrays Numpy
# ATENÇÃO: Estes são apenas vetores de strings e inteiros, não os espectrogramas!
# Isso usa MB de RAM, não GB.
all_filepaths = np.array(filepaths)
all_labels = np.array(y_data)
all_groups = np.array(groups_data)

# Shape final de entrada para a rede
# Adicionamos o '1' para o canal (channels_last)
INPUT_SHAPE = SPEC_SHAPE + (1,) 

print("\nColeta de metadados concluída.")
print(f"Total de amostras válidas: {len(all_filepaths)}")
print(f"Shape de entrada da CNN: {INPUT_SHAPE}")

Iniciando processamento e carregamento de dados...
Verificando cache em ../preprocessed_spectrograms/30s/


Processando/Verificando faixas:  58%|█████▊    | 4605/8000 [00:05<00:08, 398.25it/s]


Erro ao processar ../fma_datasets/fma_small_genres/Electronic/099134.mp3: 


Processando/Verificando faixas:  63%|██████▎   | 5069/8000 [00:05<00:04, 717.02it/s]


Erro ao processar ../fma_datasets/fma_small_genres/Rock/108925.mp3: 


Processando/Verificando faixas:  89%|████████▊ | 7096/8000 [00:07<00:01, 844.88it/s]


Erro ao processar ../fma_datasets/fma_small_genres/Experimental/133297.mp3: 


Processando/Verificando faixas: 100%|██████████| 8000/8000 [00:08<00:00, 913.32it/s] 


Coleta de metadados concluída.
Total de amostras válidas: 7997
Shape de entrada da CNN: (128, np.int64(1292), 1)


### Treino dos Modelos

In [6]:
def build_model(input_shape, num_classes):
    """
    Constrói um modelo CNN 2D estilo VGG-like.
    """
    
    # Input Layer
    inp = Input(shape=input_shape, name='input_spectrogram')
    
    # --- Bloco 1 ---
    # Usar 'padding=same' ajuda a manter as dimensões antes do MaxPooling
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_1')(inp)
    x = BatchNormalization(name='bn1_1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_2')(x)
    x = BatchNormalization(name='bn1_2')(x)
    x = MaxPooling2D((2, 2), name='pool1')(x)
    x = Dropout(0.25, name='drop1')(x)
    
    # --- Bloco 2 ---
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_1')(x)
    x = BatchNormalization(name='bn2_1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_2')(x)
    x = BatchNormalization(name='bn2_2')(x)
    x = MaxPooling2D((2, 2), name='pool2')(x)
    x = Dropout(0.25, name='drop2')(x)

    # --- Bloco 3 (Opcional, bom se a VRAM permitir) ---
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_1')(x)
    x = BatchNormalization(name='bn3_1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_2')(x)
    x = BatchNormalization(name='bn3_2')(x)
    x = MaxPooling2D((2, 2), name='pool3')(x)
    x = Dropout(0.25, name='drop3')(x)

    # --- Camadas Densas ---
    x = Flatten(name='flatten')(x)
    x = Dense(128, activation='relu', name='dense1')(x)
    x = BatchNormalization(name='bn_dense1')(x)
    x = Dropout(0.5, name='drop_dense1')(x)
    
    # --- Camada de Saída ---
    out = Dense(num_classes, activation='softmax', name='output')(x)
    
    model = Model(inputs=inp, outputs=out)
    
    # Compilar o modelo
    # Usamos 'sparse_categorical_crossentropy' pois nossos labels (y) são inteiros (0, 1, 2...)
    # Se tivéssemos usado one-hot-encoding, usaríamos 'categorical_crossentropy'
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Construir e mostrar o sumário
model = build_model(INPUT_SHAPE, N_CLASSES)
model.summary()

I0000 00:00:1763248659.633029     854 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3584 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_spectrogram (InputLayer)  │ (None, 128, 1292, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_1 (Conv2D)                │ (None, 128, 1292, 32)  │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1_1 (BatchNormalization)      │ (None, 128, 1292, 32)  │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_2 (Conv2D)                │ (None, 128, 1292, 32)  │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1_2 (BatchNormalization)      │ (None, 128, 1292, 32)  │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 64, 646, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 64, 646, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_1 (Conv2D)                │ (None, 64, 646, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2_1 (BatchNormalization)      │ (None, 64, 646, 64)    │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_2 (Conv2D)                │ (None, 64, 646, 64)    │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2_2 (BatchNormalization)      │ (None, 64, 646, 64)    │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 32, 323, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 32, 323, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_1 (Conv2D)                │ (None, 32, 323, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3_1 (BatchNormalization)      │ (None, 32, 323, 128)   │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_2 (Conv2D)                │ (None, 32, 323, 128)   │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3_2 (BatchNormalization)      │ (None, 32, 323, 128)   │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 16, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop3 (Dropout)                 │ (None, 16, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 329728)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │    42,205,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_dense1 (BatchNormalization)  │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop_dense1 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,495,080 (162.11 MB)

 Trainable params: 42,493,928 (162.10 MB)

 Non-trainable params: 1,152 (4.50 KB)

In [7]:
# --- Configuração do Treinamento ---
N_SPLITS = 10
EPOCHS = 50
BATCH_SIZE = 4 # 32 deve caber em 6GB. Se ainda quebrar, tente 16.

# Usar GroupKFold baseado no artist_id
gkf = GroupKFold(n_splits=N_SPLITS)

# Listas para armazenar resultados
fold_histories = []
fold_accuracies = []
all_y_true = []
all_y_pred = []

# --- Função Helper para o Pipeline tf.data ---

def load_and_format(filepath, label):
    """Lê o .npy, decodifica e adiciona a dimensão do canal."""
    # Usar np.load dentro de tf.py_function
    def _load_npy(path):
        return np.load(path.numpy()).astype('float32')

    spectrogram = tf.py_function(_load_npy, [filepath], tf.float32)
    
    # Definir o shape manualmente (py_function perde o shape)
    spectrogram.set_shape(SPEC_SHAPE)
    
    # Adicionar a dimensão do canal (ex: 128, 1293) -> (128, 1293, 1)
    spectrogram = tf.expand_dims(spectrogram, axis=-1)
    
    return spectrogram, label

def create_dataset(filepaths, labels):
    """Cria um pipeline tf.data otimizado."""
    
    # 1. Criar dataset de (caminhos, labels)
    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    
    # 2. Embaralhar (shuffle)
    # Buffer de 1024 arquivos. Não usa muita RAM.
    ds = ds.shuffle(buffer_size=1024)
    
    # 3. Carregar e formatar os arquivos em paralelo
    # AUTOTUNE decide quantos threads usar
    ds = ds.map(load_and_format, num_parallel_calls=tf.data.AUTOTUNE)
    
    # 4. Criar Lotes (Batches)
    ds = ds.batch(BATCH_SIZE)
    
    # 5. Fazer prefetch do próximo lote para a GPU enquanto o atual treina
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return ds

# --- Loop de Treinamento (GroupKFold) ---

print(f"Iniciando Validação Cruzada de {N_SPLITS} folds...")
print("="*30)

# Iteramos sobre os *arrays de metadados*, não sobre X
for fold, (train_idx, test_idx) in enumerate(gkf.split(all_filepaths, all_labels, groups=all_groups)):
    print(f"\n--- FOLD {fold + 1}/{N_SPLITS} ---")
    
    tf.keras.backend.clear_session()
    model = build_model(INPUT_SHAPE, N_CLASSES)
    
    # 1. Selecionar os *caminhos* e *labels* para este fold
    train_paths, train_labels = all_filepaths[train_idx], all_labels[train_idx]
    test_paths, test_labels = all_filepaths[test_idx], all_labels[test_idx]
    
    print(f"Treino: {len(train_paths)} amostras, Teste: {len(test_paths)} amostras")
    
    # 2. Criar os pipelines de dados
    train_dataset = create_dataset(train_paths, train_labels)
    test_dataset = create_dataset(test_paths, test_labels) # Não precisa de shuffle
    
    # Callback de Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', 
                                  patience=5, 
                                  verbose=1, 
                                  restore_best_weights=True)
    
    # 3. Treinar o modelo usando os datasets
    history = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=EPOCHS,
        callbacks=[early_stopping],
        verbose=1
    )
    
    fold_histories.append(history)
    
    # 4. Avaliar
    # Precisamos recriar o test_dataset sem shuffle para a matriz de confusão
    test_dataset_no_shuffle = create_dataset(test_paths, test_labels).unbatch().batch(BATCH_SIZE) # Recria sem shuffle
    
    loss, acc = model.evaluate(test_dataset_no_shuffle, verbose=0)
    fold_accuracies.append(acc)
    print(f"Acurácia do Fold {fold + 1}: {acc:.4f}")
    
    # 5. Guardar predições
    y_pred_probs = model.predict(test_dataset_no_shuffle)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    
    all_y_true.extend(test_labels) # test_labels já está na ordem correta
    all_y_pred.extend(y_pred_classes)

print("\n="*30)
print("Treinamento concluído.")

Iniciando Validação Cruzada de 10 folds...

--- FOLD 1/10 ---
Treino: 7197 amostras, Teste: 800 amostras
Epoch 1/50


I0000 00:00:1763248663.174157    2273 service.cc:152] XLA service 0x7a2c4c0012e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763248663.174196    2273 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-11-15 19:17:43.266455: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1763248663.690126    2273 cuda_dnn.cc:529] Loaded cuDNN version 91002
2025-11-15 19:17:53.626006: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng20{k2=2,k4=1,k5=0,k6=0,k7=0,k19=0} for conv %cudnn-conv-bias-activation.22 = (f32[4,128,32,323]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,64,32,323]{3,2,1,0} %bitcast.18641, f32[128,64,3,3]{3,2,1,0} %bitcast.18632, f32[128]{0} %bitcast.18691), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_ta

   1/1800 ━━━━━━━━━━━━━━━━━━━━ 6:58:11 14s/step - accuracy: 0.0000e+00 - loss: 4.0276

I0000 00:00:1763248674.704368    2273 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1800/1800 ━━━━━━━━━━━━━━━━━━━━ 118s 58ms/step - accuracy: 0.2405 - loss: 2.0818 - val_accuracy: 0.2850 - val_loss: 2.1550
Epoch 2/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 95s 53ms/step - accuracy: 0.2657 - loss: 1.9578 - val_accuracy: 0.1800 - val_loss: 2.7006
Epoch 3/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 95s 53ms/step - accuracy: 0.2639 - loss: 1.9368 - val_accuracy: 0.3212 - val_loss: 6.9048
Epoch 4/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 95s 53ms/step - accuracy: 0.2527 - loss: 1.9536 - val_accuracy: 0.2475 - val_loss: 1.9718
Epoch 5/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 93s 51ms/step - accuracy: 0.2922 - loss: 1.8758 - val_accuracy: 0.3338 - val_loss: 1.8399
Epoch 6/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 92s 51ms/step - accuracy: 0.2579 - loss: 1.9417 - val_accuracy: 0.2637 - val_loss: 1.9976
Epoch 7/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 91s 51ms/step - accuracy: 0.2796 - loss: 1.9071 - val_accuracy: 0.2763 - val_loss: 1.9643
Epoch 8/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 91s 50ms/step - accuracy: 0.3015 - loss: 1.8

...

Acurácia do Fold 3: 0.4613
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step

--- FOLD 4/10 ---
Treino: 7197 amostras, Teste: 800 amostras
Epoch 1/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 104s 53ms/step - accuracy: 0.2634 - loss: 2.0946 - val_accuracy: 0.1300 - val_loss: 3.7297
Epoch 2/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 91s 51ms/step - accuracy: 0.3036 - loss: 1.8883 - val_accuracy: 0.1800 - val_loss: 2.3402
Epoch 3/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 91s 50ms/step - accuracy: 0.2907 - loss: 1.9054 - val_accuracy: 0.2700 - val_loss: 1.9047
Epoch 4/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 91s 50ms/step - accuracy: 0.3242 - loss: 1.8289 - val_accuracy: 0.2425 - val_loss: 2.0096
Epoch 5/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 91s 50ms/step - accuracy: 0.3363 - loss: 1.7896 - val_accuracy: 0.3025 - val_loss: 1.9165
Epoch 6/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 91s 50ms/step - accuracy: 0.3608 - loss: 1.7466 - val_accuracy: 0.3425 - val_loss: 1.7821
Epoch 7/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 91s 50ms/step - accuracy: 0.3804 -

2025-11-15 20:36:58.213577: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10252760315308008007
2025-11-15 20:36:58.213707: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 8888700794520520537


Acurácia do Fold 4: 0.3775
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step

--- FOLD 5/10 ---
Treino: 7197 amostras, Teste: 800 amostras
Epoch 1/50


2025-11-15 20:37:26.533340: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 161.00MiB (rounded to 168820736)requested by op 
2025-11-15 20:37:26.533492: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-11-15 20:37:26.533520: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 360, Chunks in use: 360. 90.0KiB allocated for chunks. 90.0KiB in use in bin. 42.8KiB client-requested in use in bin.
2025-11-15 20:37:26.533530: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 160, Chunks in use: 160. 80.8KiB allocated for chunks. 80.8KiB in use in bin. 80.0KiB client-requested in use in bin.
2025-11-15 20:37:26.533540: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (1024): 	Total Chunks: 17, Chunks in use: 17. 21.2KiB allocated for chunks. 21.2KiB in use in bin. 18.8KiB client-reque

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/home/zurua/miniconda3/envs/tcc-cnn/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/zurua/miniconda3/envs/tcc-cnn/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/zurua/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/zurua/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/zurua/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/zurua/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/home/zurua/miniconda3/envs/tcc-cnn/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/home/zurua/miniconda3/envs/tcc-cnn/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/home/zurua/miniconda

### Resultados

In [ ]:
print("\n--- Resultados Finais da Validação Cruzada (10-Folds) ---")

# 1. Acurácia Média
mean_acc = np.mean(fold_accuracies)
std_acc = np.std(fold_accuracies)
print(f"Acurácia Média: {mean_acc:.4f} (+/- {std_acc:.4f})")

# 2. Matriz de Confusão Agregada
cm = confusion_matrix(all_y_true, all_y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title(f'Matriz de Confusão (Agregada) - Acurácia: {mean_acc:.3f}')
plt.ylabel('Gênero Verdadeiro')
plt.xlabel('Gênero Previsto')
plt.show()

# 3. Relatório de Classificação
print("\n--- Relatório de Classificação (Agregado) ---")
print(classification_report(all_y_true, all_y_pred, target_names=class_names))

In [ ]:
# Plotar a média das curvas de treino/validação
plt.figure(figsize=(12, 5))

# Acurácia
plt.subplot(1, 2, 1)
all_acc = [h.history['accuracy'] for h in fold_histories]
all_val_acc = [h.history['val_accuracy'] for h in fold_histories]
# Acha o tamanho da menor época (devido ao Early Stopping)
min_epochs = min([len(h) for h in all_acc])
mean_acc = np.mean([h[:min_epochs] for h in all_acc], axis=0)
mean_val_acc = np.mean([h[:min_epochs] for h in all_val_acc], axis=0)

plt.plot(mean_acc, label='Acurácia (Treino)', color='blue')
plt.plot(mean_val_acc, label='Acurácia (Validação)', color='orange')
plt.title('Acurácia Média por Época')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
all_loss = [h.history['loss'] for h in fold_histories]
all_val_loss = [h.history['val_loss'] for h in fold_histories]
min_epochs = min([len(h) for h in all_loss])
mean_loss = np.mean([h[:min_epochs] for h in all_loss], axis=0)
mean_val_loss = np.mean([h[:min_epochs] for h in all_val_loss], axis=0)

plt.plot(mean_loss, label='Loss (Treino)', color='blue')
plt.plot(mean_val_loss, label='Loss (Validação)', color='orange')
plt.title('Loss Médio por Época')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()